# Azure services for Iot

## Summary 

An introduction to the Azure services involved in Iot Development and their respective deployment using the Azure-Python SDK


## Description

The purpose of this notebook is to give an introduction to the different Azure services that are involved in Iot Solutions, and to deploy them programatically using python and the Azure-SDK.

## Requirements

* None

## Documentation

* Resource Groups: https://docs.microsoft.com/en-us/python/api/overview/azure/resources?view=azure-python
* Iot Hub: https://docs.microsoft.com/en-us/python/api/overview/azure/iot?view=azure-python
* CosmosDB: https://docs.microsoft.com/en-us/python/api/overview/azure/cosmosdb?view=azure-python
* Data Lake Store: https://docs.microsoft.com/en-us/python/api/overview/azure/data-lake-store?view=azure-python
* Data Lake Analytics: https://docs.microsoft.com/en-us/python/api/overview/azure/data-lake-analytics?view=azure-python




In [4]:
%%bash

/usr/bin/az login

bash: line 2: /usr/bin/az: No such file or directory


In [1]:
from azure.common.client_factory import get_client_from_cli_profile
subscription_id = 'your-subscription-id' ##you cand find this in the portal


## Resource group

First of all we need to set up a resource group in order to store our services.

Resource groups provide a way to monitor, control access, provision and manage billing for collections of assets that are required to run an application, or used by a client or company department.

In [6]:
from azure.mgmt.resource import ResourceManagementClient

resource_group = 'resource-group-name' ## Resource group naming 
location = 'centralus' ## 'eastus2,northeurope,centralus,westeurope', etc..

resource_client = get_client_from_cli_profile(ResourceManagementClient)
print('Creating Resource Group...')
resource_client.resource_groups.create_or_update(resource_group, {'location': location})
print('Resource Group done')

Creating Resource Group...
Resource Group done


## Iot Hub
Azure IoT Hub is a fully managed service that enables reliable and secure bidirectional communications between millions of IoT devices and a solution back end. 

Provides multiple device-to-cloud and cloud-to-device communication options. 
These options include:
* one-way messaging, file transfer, and request-reply methods.
* Provides built-in declarative message routing to other Azure services.
* Provides a queryable store for device metadata and synchronized state information.
* Enables secure communications and access control using per-device security keys or X.509 certificates.
* Provides extensive monitoring for device connectivity and device identity management events.
* Includes device libraries for the most popular languages and platforms.
    
IoTHub: Connect, monitor, and manage billions of IoT assets
* Establish bi-directional communication with billions of IoT devices
* Authenticate per device for security-enhanced IoT solutions
* Register devices at scale with IoT Hub Device Provisioning Service
* Manage your IoT devices at scale with device management
* tend the power of the cloud to your edge device

In the next lines we create an Iot hub with default characteristics and a basic SKU.

In [7]:
from azure.mgmt.iothub import IotHubClient

iot_hub_client = get_client_from_cli_profile(IotHubClient)
resouce_name = 'iot-hub-name' 
location = 'centralus'

print('Creating Iot Hub...')
iot_hub_client.iot_hub_resource.create_or_update(
    resource_group,
    resouce_name,
    {
        'location': location,
        'subscriptionid': subscription_id,
        'resourcegroup': resource_group,
        'sku': {
            'name': 'S1',
            'capacity': 2
        },
        'properties': {
            'enable_file_upload_notifications': False,
            'operations_monitoring_properties': {
            'events': {
                "C2DCommands": "Error",
                "DeviceTelemetry": "Error",
                "DeviceIdentityOperations": "Error",
                "Connections": "Information"
            }
            },
            "features": "None",
        }
    }
).result()

print('Iot Hub Done.')

Creating Iot Hub...
Iot Hub Done.


# Data Lake Store

Azure Data Lake Store is an enterprise-wide hyper-scale repository for big data analytic workloads. Azure Data Lake enables you to capture data of any size, type, and ingestion speed in one single place for operational and exploratory analytics. Data Lake Store can store trillions of files. A single file can be larger than one petabyte in size. This makes Data Lake Store ideal for storing any type of data including massive datasets like high-resolution video, genomic and seismic datasets, medical data, and data from a wide variety of industries.

In [11]:
from azure.mgmt.datalake.store import DataLakeStoreAccountManagementClient

data_lake_store_client = get_client_from_cli_profile(DataLakeStoreAccountManagementClient)
data_lake_store_account = "Data-Lake-Store-Name"
location = 'centralus'

print('Creating Data Lake Store...')
data_lake_store_client.account.create(
    resource_group,
    data_lake_store_account,
    {
        'location': location
    },
    None,
    False
).result()
print('Data Lake Store Done.')

Creating Data Lake Store...
Data Lake Store Done.


# Data Lake Analytics

Azure Data Lake Analytics allow us to process big data jobs in seconds. There is no infrastructure to worry about because there are no servers, virtual machines, or clusters to wait for, manage, or tune. We can instantly scale the processing power, measured in Azure Data Lake Analytics Units (AU), from one to thousands for each job. 

In [12]:
from azure.mgmt.datalake.analytics.account import DataLakeAnalyticsAccountManagementClient
from azure.mgmt.datalake.analytics.account.models import DataLakeAnalyticsAccount, DataLakeStoreAccountInfo


resource_name = "Data-Lake-Analytics-Name"
location = 'centralus'
data_lake_analytics_client = get_client_from_cli_profile(DataLakeAnalyticsAccountManagementClient)

print('Creating Data Lake Analytics...')
data_lake_analytics_client.account.create(
    resource_group,
   resource_name,
    DataLakeAnalyticsAccount(
        location=location,
        default_data_lake_store_account=data_lake_store_account,
        data_lake_store_accounts=[DataLakeStoreAccountInfo(name=data_lake_store_account)]
    )
).result()
print('Data Lake Analytics Done.')

Creating Data Lake Analytics...
Data Lake Analytics Done.


## Cosmos DB

Cosmos DB is Microsoft’s globally distributed, multi-model database. It supports multiple data models, including but not limited to document, graph, key-value, table, and column-family data models. CosmosDB is able to elastically and independently scale throughput and storage on demand and worldwide

CosmosDB provides a solution for doing hot path analytics of our Iot generated data.


In [13]:
from azure.mgmt.cosmosdb.cosmos_db import CosmosDB
from azure.mgmt.cosmosdb.models import Location
resource_name = "CosmosDB-Name"
location = 'centralus'
## As stated before  cosmosDB is globally distributed, therefore we can especify
## in which regions is it going to be deployed
locations = [Location( location_name= 'centralus', failover_priority = 1), Location(location_name= 'eastus', failover_priority = 0)]
cosmosdb_client = get_client_from_cli_profile(CosmosDB)


print('Creating CosmosDB...')
cosmosdb_result = cosmosdb_client.database_accounts.create_or_update(
    resource_group,
    resource_name,
    {
        'location' : location,
        'locations' : locations,
        'kind': 'MongoDB',
    }  
).result()
print('CosmosDB Done.')

Creating CosmosDB...
Creating CosmosDB...


Now if you go into the Azure Portal your services are going to be set up in the resource group that we created in the beginning 